In [13]:
from bs4 import BeautifulSoup
import requests
import re
import mysql.connector
import datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
conn = mysql.connector.connect(
    host = "*****",
    port = "****",
    user = "****",
    password = "****",
    database = "****"
)

cursor = conn.cursor(buffered=True)

수집데이터: 
*(공고 제목, 공고url)*, 주요업무, 우대사항, 근무환경, 복리후생, *(공고게시일)*
*(회사sid, 회사명, 회사형태, 산업)*, 사원수, *(설립일)*, 매출액, *(회사홈페이지)*, 평균연봉

In [17]:
begin_url = "https://www.saramin.co.kr/zf_user/search?search_area=main&search_done=y&search_optional_item=n&searchType=search&searchword=%EB%A1%9C%EB%B4%87+%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4&recruitPage={}&recruitSort=reg_dt&recruitPageCount=40&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=y"
post_base_url = "https://www.saramin.co.kr/zf_user/jobs/view?rec_idx="


session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504], allowed_methods=["GET"])
session.mount('https://', HTTPAdapter(max_retries=retries))

try: 
    res = session.get(begin_url.format(1), headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')

    cnt = re.search(r'\d+', soup.find("section", id="recruit_info").find("span", class_="cnt_result").string).group()
    index_num = int(int(cnt) / 40) + 1

    for i in range(1,index_num+1):
        try:
            res = session.get(begin_url.format(i), headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
            soup = BeautifulSoup(res.text, "html.parser")

            # i 페이지 공고id들
            recruit_info_list = soup.find("div", id="recruit_info_list").find_all("div", class_=re.compile(r"item_recruit"))

            for recruit in recruit_info_list:
                # 공고url
                post_url = post_base_url + recruit.get("value")
                print(recruit.get("value"))

                try: 
                    # jobpost enter
                    res = session.get(post_url, headers={'User-Agent': 'Mozilla/5.0'}, verify=False)
                    soup = BeautifulSoup(res.text, "html.parser")
                    # 맨 위 div; 회사명, 회사sid, 공고제목
                    jv_header = soup.find("div", class_="jv_header")
                    comp_sid = jv_header.find("div", class_="title_inner").find("button", title="관심기업 등록").get("csn")
                    comp_name = jv_header.find("div", class_="title_inner").find("a", class_="company").get("title")
                    post_title = jv_header.find("h1", class_="tit_job").string.strip().replace("\n"," ")
                    print(comp_sid, comp_name, post_title)

                    # 공고 게시일 있는 div
                    info_period = soup.find("dl", class_="info_period")
                    pd = info_period.find("dt", string="시작일").find_next_sibling("dd").get_text()
                    posted_date = datetime.datetime.strptime(pd, "%Y.%m.%d %H:%M")
                    print(posted_date)

                    # 회사형태, 산업, 설립일, 회사홈페이지 있는 div
                    info = soup.find("div", class_="wrap_info")
                    
                    comp_form = sector = establish = comp_homepage = None

                    if info:            
                        for dl in info.find("div", class_="info_area").find_all("dl"):
                            dt_text = dl.find("dt").get_text(strip=True)
                            if "기업형태" in dt_text:
                                comp_form = dl.find("dd").get("title")
                            elif "업종" in dt_text:
                                sector = dl.find("dd").get_text(strip=True)
                                print(sector)
                            elif "설립일" in dt_text:
                                ed = dl.find('dd').contents[0].strip()
                                print(ed)
                                try:
                                    establish = datetime.datetime.strptime(ed, "%Y년 %m월 %d일")
                                    print(establish)
                                except ValueError:
                                    print("ValueError")
                                    establish = None
                            elif "홈페이지" in dt_text:
                                comp_homepage = dl.find("dd").get("title")
                                print(comp_homepage)
                    else:
                        pass
                    
                    cursor.execute("""select comp_id from company where comp_sid=%s""", (comp_sid,))
                    comp_id = cursor.fetchone()

                    if comp_id:
                        comp_id = comp_id[0]            
                    else:
                        cursor.execute("""insert into company 
                                (comp_sid, comp_name, comp_form, sector, establish, comp_url)
                                values
                                (%s,%s,%s,%s,%s,%s)""",
                                (comp_sid, comp_name, comp_form, sector, establish, comp_homepage))
                        cursor.execute("""select comp_id from company where comp_sid=%s""", (comp_sid,))
                        comp_id = cursor.fetchone()[0]

                    cursor.execute("""insert into recruit
                                (comp_id, saramjob_id, post_title, post_url, posted_date)
                                values (%s, %s,%s,%s,%s)""",
                                (comp_id, 0, post_title, post_url, posted_date))
                except requests.exceptions.RequestException as e:
                    print(f"Failed to fetch job post: {post_url} due to {e}")
        except requests.exceptions.RequestException as e:
            print(f"failed to fetch page {i}: {e}")
except requests.exceptions.RequestException as e:
    print(f"Initial request failed: {e}")


/home/cook/venv/eda_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.saramin.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/cook/venv/eda_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.saramin.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


49097646


/home/cook/venv/eda_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.saramin.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


MXVieWN6VWZqMzF4RHE1Sk5pQVFPdz09 (주)비비파트너스 [SW 서비스기획 PM] 기계제조업 대기업 그룹 IT 계열사
2024-09-28 10:00:00
경영 컨설팅업
2004년 12월 28일
2004-12-28 00:00:00
www.bbp.co.kr
49097640


/home/cook/venv/eda_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.saramin.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

M25Lc3NXbjNUU0hKT0RORHVFU1pYUT09 주식회사복주 제품개발 지원 및 품질인증 부문 연구원 모집
2024-08-26 00:00:00
금속탱크 및 저장용기 제조업
2009년 7월 16일
2009-07-16 00:00:00
www.bokju.com
48731954


/home/cook/venv/eda_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.saramin.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


WlRna3gyNWVRWE9HNTFSZDA4VFYyQT09 (주)라스테크 [로보틱스부분]자율주행로봇 SW개발자
2024-08-02 00:00:00
응용 소프트웨어 개발 및 공급업
2001년 1월 3일
2001-01-03 00:00:00
http://www.rastech.co.kr
45110080


/home/cook/venv/eda_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.saramin.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


NWpoc1NKRzFmWmEzcmxhZmtyT2FmZz09 시와소프트(주) 윈도우기반 응용프로그래머 개발자 모집
2023-02-21 00:00:00
응용 소프트웨어 개발 및 공급업
2017년 12월 19일
2017-12-19 00:00:00


In [18]:
conn.commit()

In [12]:
conn.close()

In [14]:
cursor.execute("select recruit_id, post_url from recruit")
results = cursor.fetchall()

for recruit_id, post_url in results:
    res = requests.get(post_url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.text, "html.parser")
    
    info_period = soup.find("dl", class_="info_period")
    if info_period:
        pd = info_period.find("dt", string="시작일").find_next_sibling("dd").get_text()
        posted_date = datetime.datetime.strptime(pd, "%Y.%m.%d %H:%M")
        print(posted_date)
        cursor.execute("""update recruit set posted_date=%s where recruit_id=%s""",
                   (posted_date, recruit_id))
    else:
        print(f"No info_period found for {post_url}")


2024-09-27 00:00:00
2024-09-27 11:00:00
2024-09-27 11:00:00
2024-09-27 00:00:00
2024-09-27 10:00:00
2024-09-27 10:00:00
2024-09-27 09:00:00
2024-09-27 09:00:00
2024-09-27 09:00:00
2024-09-27 09:00:00
2024-09-27 09:00:00
2024-09-23 00:00:00
2024-09-27 09:00:00
2024-09-27 08:00:00
2024-09-27 08:00:00
2024-09-26 19:00:00
2024-09-26 18:00:00
2024-09-26 17:00:00
2024-09-26 17:00:00
2024-09-26 17:00:00
2024-09-27 00:00:00
2024-09-27 00:00:00
2024-09-27 00:00:00
2024-09-26 16:00:00
2024-09-26 15:00:00
2024-09-26 15:00:00
2024-09-26 14:00:00
2024-09-26 13:00:00
2024-09-26 13:00:00
2024-09-26 13:00:00
2024-09-26 13:00:00
2024-09-26 11:00:00
2024-09-26 11:00:00
2024-09-26 11:00:00
2024-09-26 10:00:00
2024-09-26 07:00:00
2024-09-25 23:00:00
2024-09-25 18:00:00
2024-09-25 17:00:00
2024-09-25 00:00:00
2024-09-25 17:00:00
2024-09-25 00:00:00
2024-09-25 15:00:00
2024-09-25 14:00:00
2024-09-25 14:00:00
2024-09-25 13:00:00
2024-09-25 12:00:00
2024-09-25 12:00:00
2024-09-25 11:00:00
2024-09-25 11:00:00
